In [1]:
import sys
import os
sys.path.append('D:\\PostGraduate\\课程\\机器学习\\optionml')
sys.path

['D:\\PostGraduate\\课程\\机器学习\\optionml\\OptionML\\simulation',
 'D:\\PostGraduate\\期权\\code\\gamma_scalping_gitlab',
 'c:\\python38\\python38.zip',
 'c:\\python38\\DLLs',
 'c:\\python38\\lib',
 'c:\\python38',
 '',
 'C:\\Users\\qiancheng\\AppData\\Roaming\\Python\\Python38\\site-packages',
 'c:\\python38\\lib\\site-packages',
 'c:\\python38\\lib\\site-packages\\win32',
 'c:\\python38\\lib\\site-packages\\win32\\lib',
 'c:\\python38\\lib\\site-packages\\Pythonwin',
 'c:\\python38\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\qiancheng\\.ipython',
 'D:\\PostGraduate\\课程\\机器学习\\optionml']

In [2]:
from OptionML.DataQry.DataGet import *
from datetime import *
import random

dt = datetime(2020, 5, 25, 14, 15, 0)
sample_size = 2000
sample_freq = 60

#获得sample_size个数据点，其中采样频率为sample_freq分钟，最后记录时间为dt
close = DataGet.get_underlying_price_ls(dt, sample_size, sample_freq)
#蒙特卡洛方法从正态分布中抽样
rand_norm = np.random.normal(0, 1, sample_size)
sim_df = pd.DataFrame(close)

trading_date_ls = [datetime.strptime(trading_date, '%Y-%m-%d %H:%M:%S') for trading_date in sim_df.index.values]
#从历史数据中计算股票的年化收益，年化波动作为漂移率和方差率
sim_df['return'] = (sim_df['close'] - sim_df['close'].shift(1)) / sim_df['close'].shift(1)
sim_df['drift'] = DataGet.get_annualized_return_ls(trading_date_ls)
sim_df['variance'] = DataGet.get_annualized_var_ls(trading_date_ls)

#计算未来一周的股价变化
dt = 5 / 250
close_chg = sim_df['drift'] * sim_df['close'] * dt + sim_df['variance'] * sim_df['close'] * rand_norm * dt ** 0.5
sim_df['sim_close'] = sim_df['close'] + close_chg

In [8]:
from pyecharts.charts import Line
import pyecharts.options as opts

start_idx = 60 * 4 * 5
future_close = sim_df['sim_close'].iloc[:-start_idx]
true_close = sim_df['close'].iloc[start_idx:]


line = Line()
line.add_xaxis(true_close.index.values.tolist())
line.add_yaxis("真实收盘价", true_close)
line.add_yaxis("模拟收盘价", future_close, is_smooth=True)
line.set_global_opts(
    yaxis_opts=opts.AxisOpts(
        is_scale=True
    )
)
line.render_notebook()

In [9]:
sim_df['close'].values

array([2.51 , 2.52 , 2.534, ..., 2.727, 2.738, 2.735])

In [10]:
#模拟lnS未来一周的走势
sim_df['lnS'] = np.log(sim_df['close'])
T = 5 / 250
sim_df['sim_lnS'] = np.zeros(sample_size)
for i in range(sample_size):
    lnS_0 = sim_df['lnS'].iloc[i]
    drift = sim_df['drift'].iloc[i]
    var = sim_df['variance'].iloc[i] ** 2
    u = lnS_0 + (drift - (var * 0.5)) * T
    sigma = var * T ** 0.5
    sim_df['sim_lnS'].iloc[i] = np.random.normal(u, sigma, 1)[0]

In [11]:

start_idx = 60 * 4 * 5
future_close = sim_df['sim_lnS'].iloc[:-start_idx]
true_close = sim_df['lnS'].iloc[start_idx:]

line = Line()
line.add_xaxis(true_close.index.values.tolist())
line.add_yaxis("真实对数价格", true_close)
line.add_yaxis("模拟对数价格", future_close, is_smooth=True)
line.set_global_opts(
    yaxis_opts=opts.AxisOpts(
        is_scale=True
    )
)
line.render_notebook()

In [6]:
future_close

2018-05-07 09:45:00    0.929488
2018-05-07 10:45:00    0.935481
2018-05-07 13:15:00    0.933426
2018-05-07 14:15:00    0.933340
2018-05-08 09:45:00    0.954118
                         ...   
2019-02-28 09:45:00    0.960878
2019-02-28 10:45:00    0.980599
2019-02-28 13:15:00    0.961299
2019-02-28 14:15:00    0.967279
2019-03-01 09:45:00    0.975549
Name: sim_lnS, Length: 801, dtype: float64

In [7]:
true_close

2019-07-25 14:15:00    1.057790
2019-07-26 09:45:00    1.058831
2019-07-26 10:45:00    1.059178
2019-07-26 13:15:00    1.059871
2019-07-26 14:15:00    1.061948
                         ...   
2020-05-22 14:15:00    1.008688
2020-05-25 09:45:00    1.005766
2020-05-25 10:45:00    1.003202
2020-05-25 13:15:00    1.007228
2020-05-25 14:15:00    1.006131
Name: lnS, Length: 801, dtype: float64